***
### Plan:
- LeNet-5
- Playing around with my own
- One(s) found on Kaggle
### Presentation:
- Explain how CNNs work (convolution, pooling, filters/feature maps, etc.)
- Explain LeNet-5 architecture
- Show code and results from 3 different tests
***

In [22]:
import torch

has_mps = torch.backends.mps.is_built()
device = "mps" if torch.backends.mps.is_built() else "cpu"

print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

MPS (Apple Metal) is AVAILABLE
Target device is mps


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

`In terms of dataframes:`
>`0 = train`<br>
>`1 = test`

***
`LOAD DATA`
***

In [2]:
data_dir = 'data'
df0 = pd.read_csv(f'{data_dir}/mnist_train.csv')
training_input = df0.drop(columns='label').to_numpy()
training_output = df0['label'].to_numpy()
df0.head

<bound method NDFrame.head of        label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0          5    0    0    0    0    0    0    0    0    0  ...      0      0   
1          0    0    0    0    0    0    0    0    0    0  ...      0      0   
2          4    0    0    0    0    0    0    0    0    0  ...      0      0   
3          1    0    0    0    0    0    0    0    0    0  ...      0      0   
4          9    0    0    0    0    0    0    0    0    0  ...      0      0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
59995      8    0    0    0    0    0    0    0    0    0  ...      0      0   
59996      3    0    0    0    0    0    0    0    0    0  ...      0      0   
59997      5    0    0    0    0    0    0    0    0    0  ...      0      0   
59998      6    0    0    0    0    0    0    0    0    0  ...      0      0   
59999      8    0    0    0    0    0    0    0    0    0  ...      0      0   

       28

In [3]:
df1 = pd.read_csv(f'{data_dir}/mnist_test.csv')
testing_input = df1.drop(columns='label').to_numpy()
testing_output = df1['label'].to_numpy()
df1.head

<bound method NDFrame.head of       label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0         7    0    0    0    0    0    0    0    0    0  ...      0      0   
1         2    0    0    0    0    0    0    0    0    0  ...      0      0   
2         1    0    0    0    0    0    0    0    0    0  ...      0      0   
3         0    0    0    0    0    0    0    0    0    0  ...      0      0   
4         4    0    0    0    0    0    0    0    0    0  ...      0      0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
9995      2    0    0    0    0    0    0    0    0    0  ...      0      0   
9996      3    0    0    0    0    0    0    0    0    0  ...      0      0   
9997      4    0    0    0    0    0    0    0    0    0  ...      0      0   
9998      5    0    0    0    0    0    0    0    0    0  ...      0      0   
9999      6    0    0    0    0    0    0    0    0    0  ...      0      0   

      28x21  28x22  2

***
`CHECK COLUMNS`
***

In [4]:
df0.columns

Index(['label', '1x1', '1x2', '1x3', '1x4', '1x5', '1x6', '1x7', '1x8', '1x9',
       ...
       '28x19', '28x20', '28x21', '28x22', '28x23', '28x24', '28x25', '28x26',
       '28x27', '28x28'],
      dtype='object', length=785)

In [5]:
df1.columns

Index(['label', '1x1', '1x2', '1x3', '1x4', '1x5', '1x6', '1x7', '1x8', '1x9',
       ...
       '28x19', '28x20', '28x21', '28x22', '28x23', '28x24', '28x25', '28x26',
       '28x27', '28x28'],
      dtype='object', length=785)

***
`TEST THAT TRAINING AND TESTING DATA EXTRACTED CORRECTLY`
***

In [6]:
all(df0['label'] == training_output)

True

In [7]:
all(df1['label'] == testing_output)

True

In [8]:
all(df0.drop(columns='label') == training_input)

True

In [9]:
all(df1.drop(columns='label') == testing_input)

True

In [10]:
training_input = training_input/255
testing_input = testing_input/255

***
`CHANGE TESTING DATA TO BINARY CLASS MATRIX AND TEST THAT IT IS CORRECT`
***

In [11]:
def to_categorical(ary):
    new = []
    for i in range(len(ary)):
        new.append([0 for x in range(ary[i])] + [1] + [0 for x in range(10-ary[i]-1)])
    return np.array(new)

In [12]:
training_output = to_categorical(training_output)
i = 2
all(np.where(training_output == 1)[1] == df0['label'])

True

In [13]:
testing_output = to_categorical(testing_output)
i = 2
all(np.where(testing_output == 1)[1] == df1['label'])

True

In [14]:
m = training_input[0][0]
for i in training_input:
    if max(i) > m:
        m = max(i)
print(m)

1.0


In [15]:
m = testing_input[0][0]
for i in testing_input:
    if max(i) > m:
        m = max(i)
print(m)

1.0


***
`SETTING UP NEURAL NETWORKS`
***

## LeNet-5

In [29]:
# GitHub: aladdinpersson

import torch
import torch.nn as nn


class LeNet5(nn.Module):
    def __inint__(self):
        super(LeNet5, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0)
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        
        return x


In [ ]:
# GitHub: aladdinpersson

import torch
import torch.nn as nn


class LeNet5(nn.Module):
    def __inint__(self):
        super(LeNet5, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0),
            nn.ReLU()
        )
        # self.relu = nn.ReLU()
        # self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        # self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        # self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        # self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0)
        
        self.lin = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

        # self.linear1 = nn.Linear(120, 84)
        # self.linear2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv(x)
        # x = torch.flatten(x, 1)
        x = x.reshape(x.shape[0], -1)
        x = self.lin(x)

        x = F.softmax(x, dim=1)

        # x = self.relu(self.conv1(x))
        # x = self.pool(x)
        # x = self.relu(self.conv2(x))
        # x = self.pool(x)
        # x = self.relu(self.conv3(x))
        # x = x.reshape(x.shape[0], -1)
        # x = self.relu(self.linear1(x))
        # x = self.linear2(x)
        
        return x


## My Take

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
transform = transforms.ToTensor()
batch_size = 8
train = torchvision.datasets.mnist(root='')